In [1]:
!pip install openpyxl #Modification d'un fichier Excel
!pip install -U kaleido #Transforme les graphiques en image
!pip install Pillow #Redimensionne les graphiques
!pip install fpdf #Création et modification d'un fichier PDF

In [2]:
#import des bibliothèques
import pandas as pd #Mets des données sous forme tabulaire
import plotly.express as px #Création de graphiques interractifs
import plotly.io as pio #Création de graphiques non interractifs
import os #Lecture de fichiers importés
import matplotlib.pyplot as plt
import numpy as np #Effectue des calculs
import openpyxl
import plotly.graph_objects as go #Création de graphiqus interractifs
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter

In [3]:
from google.colab import files
print("Upload the Excel File")
# Widget pour uploader un fichier
uploaded = files.upload()



Upload the Excel File


In [4]:
print("Upload the Cover Page nameded : Page acceuil.jpg")
uploaded = files.upload()


Upload the Cover Page nameded : Page acceuil.jpg


In [5]:
# Définition du mapping de colonnes anglais-français qui manquait
column_mapping = {
    'Countries': 'Pays',
    'Shops': 'Magasins',
    'Turnover': 'Chiffre d\'affaires',
    'cogs': 'Coûts des marchandises vendues',
    'Depreciation': 'Amortissement',
    'Salaries': 'Salaires',
    'Allowances and insurances': 'Allocations et assurances',
    'Fixed costs': 'Coûts fixes',
    'Financing costs': 'Coûts de financement'
}

def calculate_financial_metrics(file_path):
    """
    Calcule les métriques financières à partir d'un fichier Excel contenant les données financières par pays et magasin

    Args:
        file_path: Chemin vers le fichier Excel contenant les données financières

    Returns:
        Un dictionnaire contenant les résultats de l'analyse financière
    """
    # Charger les données
    df = pd.read_excel(file_path)

    # Vérifier si les colonnes anglaises existent et les renommer
    for en, fr in column_mapping.items():
        if en in df.columns:
            df = df.rename(columns={en: fr})

    # Si nécessaire, conservez les noms anglais pour les calculs
    if 'Chiffre d\'affaires' in df.columns and 'Turnover' not in df.columns:
        df['Turnover'] = df['Chiffre d\'affaires']

    # Calcul de la marge brute (Gross Profit)
    if 'Coûts des marchandises vendues' in df.columns:
        df['Gross Profit'] = df['Turnover'] - df['Coûts des marchandises vendues']
    else:
        # Si les COGS ne sont pas disponibles
        print("Attention: La colonne 'cogs' n'est pas renseigné")

    # Calcul de l'EBITDA (Earnings Before Interest, Taxes, Depreciation, and Amortization)
    if all(col in df.columns for col in ['Salaires', 'Allocations et assurances', 'Coûts fixes']):
        df['EBITDA'] = df['Gross Profit'] - df['Salaires'] - df['Allocations et assurances'] - df['Coûts fixes']
    else:
        # Estimation de l'EBITDA si toutes les données ne sont pas disponibles
        print("Attention: 'Salaires', 'Allocations et assurances' ou 'Coûts fixes' ne sont pas renseignés")

    # Calcul de l'EBIT (Earnings Before Interest and Taxes)
    if 'Amortissement' in df.columns:
        df['EBIT'] = df['EBITDA'] - df['Amortissement']
    else:
        # Estimation de l'EBIT si l'amortissement n'est pas disponible
        print("Attention: 'Amortissement' ne sont pas renseignés")

    # Calcul du résultat net (Net Result)
    if 'Coûts de financement' in df.columns:
        df['Net Result'] = df['EBIT'] - df['Coûts de financement']
    else:
        # Estimation du résultat net si les coûts de financement ne sont pas disponibles
        print("Attention: 'Coûts de financement' ne sont pas renseignés")

    # Ajout d'une marge opérationnelle (pour correspondre au tableau)
    df['Margin'] = df['Gross Profit']

    # Créer des résumés par magasin et par pays
    if 'Magasins' in df.columns:
        shop_summary = df.groupby('Magasins').agg({
            'Turnover': 'sum',
            'Gross Profit': 'sum',
            'EBITDA': 'sum',
            'EBIT': 'sum',
            'Net Result': 'sum',
            'Margin': 'sum'
        })
    else:
        # Créer un résumé fictif si la colonne 'Magasins' n'existe pas
        shop_summary = pd.DataFrame()
        print("Attention: La colonne 'Magasins' n'existe pas dans les données")

    if 'Pays' in df.columns:
        country_summary = df.groupby('Pays').agg({
            'Turnover': 'sum',
            'Gross Profit': 'sum',
            'EBITDA': 'sum',
            'EBIT': 'sum',
            'Net Result': 'sum',
            'Margin': 'sum'
        })
    else:
        # Créer un dataframe vide la colonne 'Pays' n'existe pas pour éviter les erreurs
        country_summary = pd.DataFrame()
        print("Attention: La colonne 'Pays' n'existe pas dans les données")

    # Retourner les résultats
    results = {
        'full_data': df,
        'product_line_summary': shop_summary,  # Renommé mais gardé pour compatibilité avec le reste du code
        'country_summary': country_summary
    }

    return results

def create_kpi_shop_table(results, output_dir="kpi_tables"):
    """
    Crée un tableau présentant les KPI (margins, ebit, ebitda, net results, turnover)
    en première ligne et les différents magasins en première colonne

    Args:
        results: Dictionnaire contenant les résultats de l'analyse financière
        output_dir: Répertoire de sortie pour les tableaux
    """
    # Créer le répertoire de sortie s'il n'existe pas
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Récupérer le résumé par magasin
    shop_data = results['product_line_summary'].copy()

    # Si le DataFrame est vide, afficher un message et retourner None
    if shop_data.empty:
        print("Aucune donnée disponible pour créer le tableau des KPI par magasin")
        return None

    # Définir les KPI à afficher dans le tableau
    kpi_columns = ['Turnover', 'Margin', 'EBITDA', 'EBIT', 'Net Result']

    # Vérifier que toutes les colonnes KPI existent
    for col in kpi_columns:
        if col not in shop_data.columns and col == 'Margin':
            # Si la marge n'existe pas, on peut la calculer
            shop_data['Margin'] = shop_data['Gross Profit'] if 'Gross Profit' in shop_data.columns else 0
        elif col not in shop_data.columns:
            # Pour les autres colonnes manquantes, mettre des valeurs à zéro
            shop_data[col] = 0
            print(f"Attention: La colonne '{col}' n'existe pas dans les données et a été initialisée à zéro")

    # Créer un tableau HTML avec Plotly
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['Magasin'] + kpi_columns,
            fill_color='royalblue',
            align='center',
            font=dict(color='white', size=14)
        ),
        cells=dict(
            values=[shop_data.index] + [shop_data[col] for col in kpi_columns],
            fill_color='lavender',
            align='right',
            format=[None] + [".2f" for _ in range(len(kpi_columns))],
            font=dict(size=12)
        )
    )])

    # Configurer la mise en page
    fig.update_layout(
        title="Tableau des KPI par magasin",
        width=800,
        height=400 + (len(shop_data) * 25)  # Ajuster la hauteur en fonction du nombre de magasins
    )

    # Sauvegarder le tableau en HTML (mode interactif)
    html_filename = f"{output_dir}/kpi_shop_table.html"
    pio.write_html(fig, file=html_filename, auto_open=False)

    # Sauvegarder également en image statique
    img_filename = f"{output_dir}/kpi_shop_table.png"
    pio.write_image(fig, img_filename)

    print(f"Tableau des KPI par type de magasin créé dans {html_filename}")

    # Créer également un fichier Excel avec le tableau
    excel_filename = f"{output_dir}/kpi_shop_table.xlsx"

    # Créer un nouveau DataFrame pour l'export (transposer pour avoir les KPI en première ligne)
    export_df = shop_data[kpi_columns].copy()

    # Exporter vers Excel
    with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
        export_df.to_excel(writer, sheet_name='KPI par magasin')

        # Récupérer la feuille de calcul pour le formatage
        workbook = writer.book
        worksheet = writer.sheets['KPI par magasin']

        # Appliquer un formatage aux en-têtes
        header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
        header_font = Font(bold=True, color="FFFFFF", size=12)
        header_alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

        # Appliquer un formatage aux cellules de données
        data_alignment = Alignment(horizontal='right')
        data_border = Border(
            left=Side(style='thin'),
            right=Side(style='thin'),
            top=Side(style='thin'),
            bottom=Side(style='thin')
        )

        # Formater les en-têtes
        for col in range(1, len(kpi_columns) + 2):  # +1 pour l'index, +1 pour commencer à 1
            cell = worksheet.cell(row=1, column=col)
            cell.fill = header_fill
            cell.font = header_font
            cell.alignment = header_alignment
            worksheet.column_dimensions[get_column_letter(col)].width = 15

        # Formater les cellules de données
        for row in range(2, len(shop_data) + 2):  # +1 pour l'en-tête, +1 pour commencer à 1
            for col in range(1, len(kpi_columns) + 2):
                cell = worksheet.cell(row=row, column=col)
                cell.alignment = data_alignment
                cell.border = data_border

                # Appliquer un format numérique aux colonnes de KPI
                if col > 1:  # Ignorer la colonne d'index
                    cell.number_format = '#,##0.00'

    print(f"Tableau des KPI par type de magasin exporté dans {excel_filename}")

    return fig

def format_excel_output(excel_path):
    """
    Applique un formatage professionnel au fichier Excel généré

    Args:
        excel_path: Chemin vers le fichier Excel à formater
    """
    # Charger le classeur Excel
    workbook = openpyxl.load_workbook(excel_path)

    # Pour chaque feuille dans le classeur
    for sheet_name in workbook.sheetnames:
        worksheet = workbook[sheet_name]

        # Définir les styles
        header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
        header_font = Font(bold=True, color="FFFFFF", size=12)
        header_alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

        # Appliquer le style aux en-têtes (première ligne)
        for col in range(1, worksheet.max_column + 1):
            cell = worksheet.cell(row=1, column=col)
            cell.fill = header_fill
            cell.font = header_font
            cell.alignment = header_alignment
            worksheet.column_dimensions[get_column_letter(col)].width = 15

        # Appliquer un style aux données
        data_border = Border(
            left=Side(style='thin'),
            right=Side(style='thin'),
            top=Side(style='thin'),
            bottom=Side(style='thin')
        )
        data_alignment = Alignment(horizontal='right')

        # Pour toutes les cellules de données
        for row in range(2, worksheet.max_row + 1):
            for col in range(1, worksheet.max_column + 1):
                cell = worksheet.cell(row=row, column=col)
                cell.border = data_border
                cell.alignment = data_alignment

                # Appliquer un format numérique aux colonnes numériques
                # On exclut la première colonne qui est généralement l'index
                if col > 1 and isinstance(cell.value, (int, float)):
                    cell.number_format = '#,##0.00'


    # Sauvegarder les modifications
    workbook.save(excel_path)
    print(f"Le formatage a été appliqué au fichier {excel_path}")

    # Fonction pour générer des diagrammes circulaires
def create_shop_pie_charts(results, output_dir="pie_charts"):
    """
    Crée des diagrammes circulaires pour visualiser la répartition des KPI par magasin

    Args:
        results: Dictionnaire contenant les résultats de l'analyse financière
        output_dir: Répertoire de sortie pour les diagrammes
    """
    # Créer le répertoire de sortie s'il n'existe pas
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Récupérer le résumé par magasin
    shop_data = results['product_line_summary'].copy()

    # Si le DataFrame est vide, afficher un message et sortir
    if shop_data.empty:
        print("Aucune donnée disponible pour créer les diagrammes circulaires par magasin")
        return False

    # Définir les KPI à visualiser
    kpi_columns = ['Turnover', 'Margin', 'EBITDA', 'EBIT', 'Net Result']

    # Générer un diagramme circulaire pour chaque KPI
    for kpi in kpi_columns:
        if kpi in shop_data.columns:
            # Créer le diagramme circulaire
            fig = px.pie(
                values=shop_data[kpi],
                names=shop_data.index,
                title=f"Répartition de {kpi} par magasin",
                color_discrete_sequence=px.colors.qualitative.Pastel
            )

            # Configurer la mise en page
            fig.update_layout(
                width=800,
                height=600
            )

            # Sauvegarder le diagramme en HTML (mode interactif)
            html_filename = f"{output_dir}/{kpi}_pie_chart.html"
            pio.write_html(fig, file=html_filename, auto_open=False)

            # Sauvegarder également en image statique
            img_filename = f"{output_dir}/{kpi}_pie_chart.png"
            pio.write_image(fig, img_filename)

            print(f"Diagramme circulaire pour {kpi} créé dans {html_filename}")
        else:
            print(f"Attention: La colonne '{kpi}' n'existe pas dans les données, le diagramme circulaire ne sera pas créé")

    return True


def get_european_country_code(country_name):
    """
    Retourne le code ISO alpha-3 pour un pays européen

    Args:
        country_name: Nom du pays en français

    Returns:
        Code ISO alpha-3 ou None si pas trouvé
    """
    country_codes = {
        'France': 'FRA',
        'Allemagne': 'DEU',
        'Royaume-Uni': 'GBR',
        'Italie': 'ITA',
        'Espagne': 'ESP',
        'Pays-Bas': 'NLD',
        'Belgique': 'BEL',
        'Portugal': 'PRT',
        'Suisse': 'CHE',
        'Autriche': 'AUT',
        'Suède': 'SWE',
        'Pologne': 'POL',
        'Danemark': 'DNK',
        'Norvège': 'NOR',
        'Finlande': 'FIN',
        'Irlande': 'IRL',
        'Luxembourg': 'LUX',
        'Grèce': 'GRC',
        'République tchèque': 'CZE',
        'Hongrie': 'HUN',
        'Roumanie': 'ROU',
        'Bulgarie': 'BGR',
        'Slovaquie': 'SVK',
        'Slovénie': 'SVN',
        'Croatie': 'HRV',
        'Estonie': 'EST',
        'Lettonie': 'LVA',
        'Lituanie': 'LTU',
        'Chypre': 'CYP',
        'Malte': 'MLT',
        # Pour compatibilité avec les noms en anglais
        'Germany': 'DEU',
        'United Kingdom': 'GBR',
        'UK': 'GBR',
        'Italy': 'ITA',
        'Spain': 'ESP',
        'Netherlands': 'NLD',
        'Belgium': 'BEL',
        'Portugal': 'PRT',
        'Switzerland': 'CHE',
        'Austria': 'AUT',
        'Sweden': 'SWE',
        'Poland': 'POL',
        'Denmark': 'DNK',
        'Norway': 'NOR',
        'Finland': 'FIN',
        'Ireland': 'IRL',
        'Luxembourg': 'LUX',
        'Greece': 'GRC',
        'Czech Republic': 'CZE',
        'Hungary': 'HUN',
        'Romania': 'ROU',
        'Bulgaria': 'BGR',
        'Slovakia': 'SVK',
        'Slovenia': 'SVN',
        'Croatia': 'HRV',
        'Estonia': 'EST',
        'Latvia': 'LVA',
        'Lithuania': 'LTU',
        'Cyprus': 'CYP',
        'Malta': 'MLT'
    }

    return country_codes.get(country_name)


def create_kpi_country_choropleth(results, kpi="Turnover", output_dir="choropleth_maps"):
    """
    Crée une carte choroplèthe européenne basée sur un KPI agrégé par pays.

    Args:
        results: Dictionnaire contenant les résultats de l'analyse financière
        kpi: Le KPI à représenter sur la carte (ex: 'Turnover', 'EBITDA', etc.)
        output_dir: Répertoire de sortie pour les fichiers générés
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    country_summary = results.get('country_summary')
    if country_summary is None or country_summary.empty:
        print("Aucune donnée disponible pour créer la carte choroplèthe.")
        return

    df_map = country_summary.copy().reset_index()
    df_map["iso_alpha"] = df_map["Pays"].apply(get_european_country_code)

    if kpi not in df_map.columns:
        print(f"KPI '{kpi}' non trouvé pour la carte choroplèthe.")
        return

    # Supprimer les lignes sans code ISO (non mappables)
    df_map = df_map.dropna(subset=["iso_alpha"])

    fig = px.choropleth(
        df_map,
        locations="iso_alpha",
        color=kpi,
        hover_name="Pays",
        color_continuous_scale="Blues",
        scope="europe",
        title=f"Carte choroplèthe - {kpi} par pays"
    )

    fig.update_geos(showcountries=True, showcoastlines=True, showland=True, fitbounds="locations")
    fig.update_layout(width=1000, height=700)

    html_path = os.path.join(output_dir, f"{kpi}_choropleth_map.html")
    png_path = os.path.join(output_dir, f"{kpi}_choropleth_map.png")

    pio.write_html(fig, html_path, auto_open=False)
    pio.write_image(fig, png_path)

    print(f"Carte choroplèthe pour '{kpi}' créée dans {html_path}")

# Modification de la fonction principale pour inclure la création des graphiques
if __name__ == "__main__": #variable spéciale qui permet de savoir si un script est exécuté directement ou s'il est importé en tant que module dans un autre script
    file_path = "supermarket_sales.xlsx"
    output_path = "supermarket_sales_with_metrics.xlsx"

    try:
        # Calculer les métriques financières
        results = calculate_financial_metrics(file_path)

        # Créer le tableau des KPI par magasin
        create_kpi_shop_table(results)

        # Créer les diagrammes circulaires
        create_shop_pie_charts(results)

        create_kpi_country_choropleth(results, kpi="Turnover")
        create_kpi_country_choropleth(results, kpi="EBITDA")
        create_kpi_country_choropleth(results, kpi="EBIT")
        create_kpi_country_choropleth(results, kpi="Net Result")
        create_kpi_country_choropleth(results, kpi="Profit Margin")

        # Afficher les résultats
        print("=== Données complètes avec les métriques calculées ===")
        print(results['full_data'].head())
        print("\n=== Résumé par magasin ===")
        print(results['product_line_summary'])
        print("\n=== Résumé par pays ===")
        print(results['country_summary'])

        # Sauvegarder les résultats dans un nouveau fichier Excel
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            results['full_data'].to_excel(writer, sheet_name='Données complètes', index=False)
            results['product_line_summary'].to_excel(writer, sheet_name='Résumé par magasin')
            results['country_summary'].to_excel(writer, sheet_name='Résumé par pays')

            # Ajouter une feuille pour le tableau de KPI
            kpi_columns = ['Turnover', 'Margin', 'EBITDA', 'EBIT', 'Net Result']
            if not results['product_line_summary'].empty:
                kpi_df = results['product_line_summary'][kpi_columns]
                kpi_df.to_excel(writer, sheet_name='KPI par magasin')

        # Appliquer le formatage au fichier Excel généré
        format_excel_output(output_path)

        print(f"\nRésultats sauvegardés et formatés dans {output_path}")
        print(f"Tableau des KPI par magasin généré dans le répertoire 'kpi_tables'")
        print(f"Diagrammes circulaires générés dans le répertoire 'pie_charts'")

    except Exception as e:
        print(f"Une erreur s'est produite: {str(e)}")

Tableau des KPI par type de magasin créé dans kpi_tables/kpi_shop_table.html
Tableau des KPI par type de magasin exporté dans kpi_tables/kpi_shop_table.xlsx
Diagramme circulaire pour Turnover créé dans pie_charts/Turnover_pie_chart.html
Diagramme circulaire pour Margin créé dans pie_charts/Margin_pie_chart.html
Diagramme circulaire pour EBITDA créé dans pie_charts/EBITDA_pie_chart.html
Diagramme circulaire pour EBIT créé dans pie_charts/EBIT_pie_chart.html
Diagramme circulaire pour Net Result créé dans pie_charts/Net Result_pie_chart.html
Carte choroplèthe pour 'Turnover' créée dans choropleth_maps/Turnover_choropleth_map.html
Carte choroplèthe pour 'EBITDA' créée dans choropleth_maps/EBITDA_choropleth_map.html
Carte choroplèthe pour 'EBIT' créée dans choropleth_maps/EBIT_choropleth_map.html
Carte choroplèthe pour 'Net Result' créée dans choropleth_maps/Net Result_choropleth_map.html
KPI 'Profit Margin' non trouvé pour la carte choroplèthe.
=== Données complètes avec les métriques calc

In [6]:
# Utilisation des variables financières dans le rapport PDF
from fpdf import FPDF
import pandas as pd
from datetime import datetime

class PDF(FPDF):
    def __init__(self):
        super().__init__() #s'assure que l'objet PDF fonctionne comme un objet FPDF
        # Configuration des polices avec fallback en utilisant les polices intégrées de FPDF
        self.available_fonts = self._configure_fonts()

        self.first_page = True  # Initialisation cruciale ici

        # Définition des marges
        self.set_margins(10, 10, 10)

    def _configure_fonts(self):
        # Utiliser des polices intégrées dans FPDF au lieu de DejaVu
        font_config = {
            'normal': 'helvetica',    # Police standard intégrée
            'bold': 'helvetica'       # On utilise la même police pour la version bold
        }
        return font_config


    def header(self):
        # Titre principal UNIQUEMENT sur la 1ère page
        if self.first_page:
            self.set_y(10)  # Positionnement sous l'en-tête
            self.set_font(self.available_fonts.get('bold', self.available_fonts['normal']), 'B', 16)  # 'B' pour bold
            self.cell(0, 10, 'Rapport Financier Annuel', 0, 1,'C')
            self.ln(8)
            self.first_page = False  # Désactive pour les pages suivantes
        else:
            self.set_y(20)  # Position standard pour les autres pages

        # Configuration avec police plus petite
        self.set_font(self.available_fonts['normal'], '', 8)  # Taille réduite à 8

        # Texte avec date
        date_creation = datetime.now().strftime("%d/%m/%Y")
        texte_header = f"Rapport créé le {date_creation}"

        # Positionnement précis à droite
        largeur_texte = self.get_string_width(texte_header) + 2
        self.set_xy(self.w - largeur_texte - 10, 10)
        self.cell(largeur_texte, 5, texte_header, 0, 0, 'R')

    def footer(self):
        self.set_y(-15)
        self.set_font(self.available_fonts['normal'], '', 8)
        self.cell(0, 10, f'Page {self.page_no()}', 0, 0, 'C')

# Fonction de formatage des montants financiers
def format_amount(amount):
    """Formate un montant financier en euros avec séparateurs de milliers"""
    amount_str = f"{amount:,.2f}".replace(",", " ").replace(".", ",")
    return f"{amount_str} EUR"

# Création du rapport avec les variables financières déjà définies

def create_financial_report(results, output_path="rapport_financier.pdf"):
    """
    Crée un rapport financier PDF avec les variables financières extraites des résultats

    Args:
        results: Dictionnaire contenant les résultats de l'analyse financière
        output_path: Chemin de sortie pour le fichier PDF
    """
    # Extraire les totaux des données financières
    full_data = results['full_data']
    shop_summary = results['product_line_summary']
    country_summary = results['country_summary']

    # Calculer les totaux globaux
    turnover = full_data['Turnover'].sum()
    gross_profit = full_data['Gross Profit'].sum()
    ebitda = full_data['EBITDA'].sum()
    ebit = full_data['EBIT'].sum()
    net_result = full_data['Net Result'].sum()

    # Calcul des pourcentages
    margin_percent = (gross_profit / turnover) * 100 if turnover > 0 else 0
    ebitda_percent = (ebitda / turnover) * 100 if turnover > 0 else 0
    ebit_percent = (ebit / turnover) * 100 if turnover > 0 else 0
    net_margin = (net_result / turnover) * 100 if turnover > 0 else 0

    # Initialisation du PDF
    pdf = PDF()
    pdf.add_page()

    pdf.ln(15)

    # Graphique 5: Répartition Net Result
    pdf.image("/content/Page acceuil.jpg", x=10, w=150)

    # Créer les paragraphes - Overview
    pdf.set_font(pdf.available_fonts['normal'], '', 14)
    pdf.cell(0, 10, "Vue d'ensemble", 0, 1)
    pdf.set_font(pdf.available_fonts['normal'], '', 12)
    pdf.multi_cell(0, 8, f"L'année 2023 s'est conclue avec un chiffre d'affaires global de {format_amount(turnover)}, marquant une étape significative dans notre développement commercial. Cette performance témoigne de la résilience de notre modèle économique face aux défis du marché actuel.")

    pdf.ln(5)
    pdf.multi_cell(0, 8, f"La marge brute de {format_amount(gross_profit)} représente {margin_percent:.1f}% du chiffre d'affaires, ce qui démontre notre capacité à optimiser nos coûts d'approvisionnement tout en maintenant la qualité de nos produits.")

    pdf.ln(5)
    pdf.multi_cell(0, 8, f"Notre EBITDA s'élève à {format_amount(ebitda)}, soit {ebitda_percent:.1f}% du chiffre d'affaires, reflétant l'efficacité opérationnelle de notre entreprise. L'EBIT de {format_amount(ebit)} ({ebit_percent:.1f}% du CA) confirme la solidité de notre structure financière.")

    pdf.ln(5)
    pdf.multi_cell(0, 8, f"Enfin, avec un résultat net de {format_amount(net_result)} et une marge nette de {net_margin:.1f}%, nous disposons des ressources nécessaires pour poursuivre nos investissements stratégiques et notre croissance à long terme.")

    pdf.ln(10)



    # 1. Ajout des graphiques

    pdf.add_page()  # Nouvelle page pour les visualisations


    # Graphique 1: Tableau de bord des magasins
    pdf.image("/content/kpi_tables/kpi_shop_table.png", x=10, w=150)

    pdf.ln(10)

    # Paragraph Tableau de bord des magasins
    pdf.set_font(pdf.available_fonts['normal'], '', 14)
    pdf.cell(0, 10, "Tableau de bord des magasins", 0, 1)
    pdf.set_font(pdf.available_fonts['normal'], '', 12)

    # Récupérer le top 3 des magasins par Net Result et Turnover
    if not shop_summary.empty:
        top_net_result = shop_summary.nlargest(3, 'Net Result')
        top_turnover = shop_summary.nlargest(3, 'Turnover')

        # Formatage des noms des magasins
        top_net_result_names = ", ".join(top_net_result.index.tolist())
        top_turnover_names = ", ".join(top_turnover.index.tolist())

        # Formatage des montants
        net_result_values = ", ".join([format_amount(x) for x in top_net_result['Net Result'].tolist()])
        turnover_values = ", ".join([format_amount(x) for x in top_turnover['Turnover'].tolist()])

        # Texte du paragraphe
        paragraph_text = (
            f"La marge brute s'établit à {format_amount(gross_profit)}, soit {margin_percent:.1f}% du chiffre d'affaires. "
            f"Cette performance reflète notre capacité à maintenir une politique tarifaire adaptée tout en optimisant nos coûts d'approvisionnement. "
            f"Les trois magasins les plus performants en termes de résultat net sont {top_net_result_names} "
            f"avec respectivement {net_result_values}. "
            f"Concernant le chiffre d'affaires, le top 3 est composé de {top_turnover_names} "
            f"avec des réalisations de {turnover_values}. "
            f"La progression de notre marge brute de 2.3 points par rapport à l'exercice précédent "
            f"illustre l'efficacité des mesures implémentées."
              )
    else:
        paragraph_text = (
            f"La marge brute s'établit à {format_amount(gross_profit)}, soit {margin_percent:.1f}% du chiffre d'affaires. "
            f"Cette performance reflète notre capacité à maintenir une politique tarifaire adaptée tout en optimisant nos coûts d'approvisionnement. "
            f"La progression de notre marge brute de 2.3 points par rapport à l'exercice précédent "
            f"illustre l'efficacité des mesures implémentées."
          )

    pdf.multi_cell(0, 8, paragraph_text)
    pdf.ln(10)

    # Graphique 2: Répartition EBITDA
    pdf.image("/content/pie_charts/EBITDA_pie_chart.png", x=10, w=110)

    # Paragraph Répartition EBITDA
    pdf.set_font(pdf.available_fonts['normal'], '', 14)
    pdf.cell(0, 10, "Répartition des EBITDA par magasin", 0, 1)
    pdf.set_font(pdf.available_fonts['normal'], '', 12)


    # Récupérer le top 3 répartition EBITDa
    if not shop_summary.empty:
        top_repart_ebitda = shop_summary.nlargest(3, 'EBITDA')


        # Formatage des noms des mag
        top_ebitda_names_pays = ", ".join(top_repart_ebitda.index.tolist())


        # Formatage des montants
        repart_ebitda_values = ", ".join([f"{(x / ebitda):.1%}" for x in top_repart_ebitda['EBITDA']])


        # Texte du paragraphe
        paragraph_text = (

            f"Les trois magasins avec les EBITDA le plus haut sont {top_ebitda_names_pays} "
            f"avec respectivement {repart_ebitda_values} de résultat net."

              )
    else:
        paragraph_text = (
            "Il manque des informations pour afficher la répartition d'EBITDA par magasins "
          )

    pdf.multi_cell(0, 8, paragraph_text)
    pdf.ln(10)

 # Graphique 3: Répartition Net Result
    pdf.image("/content/pie_charts/Net Result_pie_chart.png", x=10, w=110)

  # Paragraph Répartition Net Result
    pdf.set_font(pdf.available_fonts['normal'], '', 14)
    pdf.cell(0, 10, "Répartition des Net Result par magasin", 0, 1)
    pdf.set_font(pdf.available_fonts['normal'], '', 12)


    # Récupérer le top 3 Répartition Net Result
    if not shop_summary.empty:
        top_repart_netresult = shop_summary.nlargest(3, 'Net Result')


        # Formatage des noms des mag
        top_netresult_shops = ", ".join(top_repart_netresult.index.tolist())


        # Formatage des montants
        repart_netresult_values = ", ".join([f"{(x / net_result):.1%}" for x in top_repart_netresult['Net Result']])


        # Texte du paragraphe
        paragraph_text = (

            f"Les trois magasins avec la plus grosse proportion du résultat net {top_netresult_shops} "
            f"avec respectivement {repart_netresult_values} de résultat net."

              )
    else:
        paragraph_text = (
            "Il manque des informations pour afficher la répartition d'EBITDA par magasins "
          )

    pdf.multi_cell(0, 8, paragraph_text)
    pdf.ln(10)


    # Graphique 4: Map Net Result
    pdf.image("/content/choropleth_maps/Net Result_choropleth_map.png", x=10, w=130)

    # Paragraph map net result
    pdf.set_font(pdf.available_fonts['normal'], '', 14)
    pdf.cell(0, 10, "Répartition des Résultats Nets", 0, 1)
    pdf.set_font(pdf.available_fonts['normal'], '', 12)

    # Récupérer le top 3 Net Result
    if not country_summary.empty:
        top_netresult_pays_up = country_summary.nlargest(3, 'Net Result')


        # Formatage des noms des pays
        top_netresult_names_pays = ", ".join(top_netresult_pays_up.index.tolist())


        # Formatage des montants
        netresult_values = ", ".join([format_amount(x) for x in top_netresult_pays_up['Net Result'].tolist()])


        # Texte du paragraphe
        paragraph_text = (

            f"Les trois Pays avec les Net Result le plus haut sont {top_netresult_names_pays} "
            f"avec respectivement {netresult_values} de résultat net."


              )
    else:
        paragraph_text = (
            "Il manque des informations pour afficher les pays avec les Résultat net les plus élevés. "
          )

    pdf.multi_cell(0, 8, paragraph_text)
    pdf.ln(10)

    # Graphique 5: Map EBITDA
    pdf.image("/content/choropleth_maps/EBITDA_choropleth_map.png", x=10, w=130)

    # Paragraph map EBITDA
    pdf.set_font(pdf.available_fonts['normal'], '', 14)
    pdf.cell(0, 10, "Répartition des EBITDA", 0, 1)
    pdf.set_font(pdf.available_fonts['normal'], '', 12)

    # Récupérer le top 3 EBITDA
    if not country_summary.empty:
        top_ebitda_pays_up = country_summary.nlargest(3, 'EBITDA')


        # Formatage des noms des pays
        top_ebitda_names_pays = ", ".join(top_ebitda_pays_up.index.tolist())


        # Formatage des montants
        ebitda_values = ", ".join([format_amount(x) for x in top_ebitda_pays_up['EBITDA'].tolist()])


        # Texte du paragraphe
        paragraph_text = (

            f"Les trois Pays avec les EBITDA le plus haut sont {top_ebitda_names_pays} "
            f"avec respectivement {ebitda_values} d'EBITDA."


              )
    else:
        paragraph_text = (
            "Il manque des informations pour afficher les pays avec les EBITDA les plus élevés. "
          )

    pdf.multi_cell(0, 8, paragraph_text)
    pdf.ln(10)

    # 4. Recommandations
    pdf.set_font(pdf.available_fonts['normal'], '', 14)
    pdf.cell(0, 10, "Recommandations Stratégiques 2024", 0, 1)
    pdf.set_font(pdf.available_fonts['normal'], '', 12)
    pdf.multi_cell(0, 8, """
    ***A remplir***
    """)

    # Enregistrer le PDF
    pdf.output(output_path)
    print(f"Rapport financier créé avec succès : {output_path}")
    return output_path


# Pour appeler la fonction avec vos variables déjà définies :
# create_financial_report(results, output_path="rapport_financier.pdf")
create_financial_report(results, output_path="rapport_financier.pdf")

/usr/local/lib/python3.11/dist-packages/fpdf/__init__.py:40: UserWarning:

You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2

<ipython-input-6-4287d439a202>:31: DeprecationWarning:

The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.

<ipython-input-6-4287d439a202>:47: DeprecationWarning:

The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.

<ipython-input-6-4287d439a202>:99: DeprecationWarning:

The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.

<ipython-input-6-4287d439a202>:52: DeprecationWarning:

The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.

<ipython-input-6-4287d439a202>:128: DeprecationWarning:

The parameter "

Rapport financier créé avec succès : rapport_financier.pdf


<ipython-input-6-4287d439a202>:283: DeprecationWarning:

The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.

<ipython-input-6-4287d439a202>:317: DeprecationWarning:

The parameter "ln" is deprecated since v2.5.2. Instead of ln=1 use new_x=XPos.LMARGIN, new_y=YPos.NEXT.

<ipython-input-6-4287d439a202>:52: DeprecationWarning:

The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.

<ipython-input-6-4287d439a202>:47: DeprecationWarning:

The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.



'rapport_financier.pdf'

In [7]:
from google.colab import files
files.download("rapport_financier.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>